In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import Concatenate,MaxPool2D
from keras.models import Model
from keras.layers import Input, Dense, Conv2D,Flatten,Dropout,AvgPool2D
import os
import cv2
import keras
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']
target_dict={k: v for v, k in enumerate(CLASSES)}
target_dict
target_val=  [target_dict[CLASSES[i]] for i in range(len(CLASSES))]

In [19]:
my_generator= ImageDataGenerator(rescale=1./255, validation_split=0.2) #rescaled the images


In [20]:
my_image_generator_train = my_generator.flow_from_directory('/content/drive/MyDrive/Data/train', #this is the image generator next we need to read the data from it
        target_size=(150, 150),
        batch_size=32,
        class_mode='sparse',
        subset="training") #does not fully load the data into the memory instead it created a channel  
validation_image_generator= my_generator.flow_from_directory('/content/drive/MyDrive/Data/train',
        target_size=(150, 150),
        batch_size=32,
        subset="validation",
        class_mode='sparse' )

Found 10243 images belonging to 104 classes.
Found 2510 images belonging to 104 classes.


In [21]:
test = my_generator.flow_from_directory(
        directory='/content/drive/MyDrive/Data/test',
        target_size=(150, 150),
        class_mode='sparse')

Found 3712 images belonging to 104 classes.


In [22]:
my_image_generator_train 

In [23]:
for batch in test:
  x_test,y_test = batch
  print ( y_test )
  break

[44. 84. 75. 88.  5.  9. 78. 22. 74.  6. 68. 62. 82. 14.  5.  5. 68. 47.
 38. 93. 74. 68. 47. 37. 50. 68. 76. 83. 74. 48.  6.  5.]


In [24]:
def googlenet(input_shape, n_classes):
  
  def inception_block(x, f):
    t1 = Conv2D(f[0], 1, activation='relu')(x)
    
    t2 = Conv2D(f[1], 1, activation='relu')(x)
    t2 = Conv2D(f[2], 3, padding='same', activation='relu')(t2)
    
    t3 = Conv2D(f[3], 1, activation='relu')(x)
    t3 = Conv2D(f[4], 5, padding='same', activation='relu')(t3)
    
    t4 = MaxPool2D(3, 1, padding='same')(x)
    t4 = Conv2D(f[5], 1, activation='relu')(t4)
    
    output = Concatenate()([t1, t2, t3, t4])
    return output
  
  
  input = Input(input_shape)
  
  x = Conv2D(64, 7, strides=2, padding='same', activation='relu')(input)
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = Conv2D(64, 1, activation='relu')(x)
  x = Conv2D(192, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(3, strides=2)(x)
  
  x = inception_block(x, [64, 96, 128, 16, 32, 32])
  x = inception_block(x, [128, 128, 192, 32, 96, 64])
  x = MaxPool2D(3, strides=2, padding='same')(x)
  
  x = inception_block(x, [192, 96, 208, 16, 48, 64])
  x = inception_block(x, [160, 112, 224, 24, 64, 64])
  x = inception_block(x, [128, 128, 256, 24, 64, 64])
  x = inception_block(x, [112, 144, 288, 32, 64, 64])
  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = MaxPool2D(3, strides=2, padding='same')(x)

  x = inception_block(x, [256, 160, 320, 32, 128, 128])
  x = inception_block(x, [384, 192, 384, 48, 128, 128])
  x = Dropout(0.4)(x)
  
  x = Flatten()(x)
  output = Dense(n_classes, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [25]:
model=googlenet((150,150,3),104)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_57 (Conv2D)             (None, 75, 75, 64)   9472        ['input_2[0][0]']                
                                                                                                  
 max_pooling2d_13 (MaxPooling2D  (None, 38, 38, 64)  0           ['conv2d_57[0][0]']              
 )                                                                                                
                                                                                            

In [26]:
from keras.optimizers import Adam
OP= Adam(lr=0.0001)
model.compile(optimizer=OP, loss=keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
#call_backs= ModelCheckpoint("f/content/drive/MyDrive/GoogleNet_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
#early = EarlyStopping(monitor='`val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(my_image_generator_train ,validation_data= validation_image_generator, epochs=10)

Epoch 1/10
321/321 [==============================] - 1911s 6s/step - loss: 4.0692 - accuracy: 0.0716 - val_loss: 3.7771 - val_accuracy: 0.1044
Epoch 2/10
321/321 [==============================] - 1510s 5s/step - loss: 3.6872 - accuracy: 0.1296 - val_loss: 3.5439 - val_accuracy: 0.1406
Epoch 3/10
321/321 [==============================] - 1509s 5s/step - loss: 3.2065 - accuracy: 0.1939 - val_loss: 2.9551 - val_accuracy: 0.2147
Epoch 4/10
321/321 [==============================] - 1508s 5s/step - loss: 2.9595 - accuracy: 0.2298 - val_loss: 2.8917 - val_accuracy: 0.2363
Epoch 5/10
321/321 [==============================] - 1521s 5s/step - loss: 2.7844 - accuracy: 0.2554 - val_loss: 2.6867 - val_accuracy: 0.2801
Epoch 6/10
321/321 [==============================] - 1512s 5s/step - loss: 2.6191 - accuracy: 0.2878 - val_loss: 2.6928 - val_accuracy: 0.2789
Epoch 7/10
321/321 [==============================] - 1491s 5s/step - loss: 2.4630 - accuracy: 0.3206 - val_loss: 2.5314 - val_accuracy:

In [ ]:
googlenet= model.save('/content/drive/MyDrive/googlenet3')

##Evaluation

In [ ]:
score = model.evaluate(test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 4.6445770263671875
Test accuracy: 0.0


In [ ]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(test, batch_size=32)
print("test loss, test acc:", results)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Evaluate on test data
116/116 [==============================] - 120s 1s/step - loss: 4.6446 - accuracy: 0.0000e+00
test loss, test acc: [4.644576549530029, 0.0]
Test loss: 4.6445770263671875 / Test accuracy: 0.0


In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
plt.show()

In [ ]:

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("Epoch")
plt.legend(["loss","Validation Loss"])
plt.show()

In [ ]:
plt.plot(hist.history["accuracy"])
plt.plot(hist.history['val_accuracy'])
plt.title("model accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Accuracy","Validation Accuracy"])
plt.show()

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(test,steps = nb_samples)
print("predictions shape:", predictions.shape)

In [ ]:
# Defining function for confusion matrix plot
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    print(cm)

    # Print Confusion matrix
    fig, ax = plt.subplots(figsize=(4, 4))
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

In [ ]:
# Making prediction
testX_arg = np.argmax(testY, axis=1)
y_pred = np.argmax(model.predict(testX), axis=1)
y_true = np.argmax(testY, axis=1)
print(y_pred)
print(y_pred.shape)

# Plotting the confusion matrix
confusion_mtx = confusion_matrix(y_true, y_pred)

CLASS_NAMES = [f.name for f in os.scandir(imgPath) if f.is_dir()]
class_names = CLASS_NAMES
print(class_names)

print("ypred\n", y_pred)
print("ytrue", y_true)

In [ ]:
testy = my_generator.flow_from_directory(
        directory='/content/drive/MyDrive/Data/test',
        target_size=(150, 150),
        batch_size=16474,
        class_mode='sparse')


for X_test, Y_test in testy:
   break